In [320]:
from shutil import copyfile
import os.path

def get_packetlog_columns():
    return [ 'extended_seq',
             'tracked_seq',
             'state',
             'tracked_ntp',
             'marker',
             'header_size',
             'payload_size',
             'timestamp',
             'payload_type',
             'played_out'];

def get_delta_packetlog_columns():
    return ["owd",
            "BiF",
            "playout_delay"];

def get_statslog_columns():
    return [ 'rcved_tot_packets',
             'rcved_tot_bytes',
             'rcved_acc_bytes',
             'rcved_acc_packets',                             

             'lost_tot_packets',
             'lost_tot_bytes',
             'lost_acc_bytes',
             'lost_acc_packets',

             'discarded_tot_packets',
             'discarded_tot_bytes',
             'discarded_acc_bytes',
             'discarded_acc_packets',

             'corrupted_tot_packets',
             'corrupted_tot_bytes',
             'corrupted_acc_bytes',
             'corrupted_acc_packets',

             'repaired_tot_packets',
             'repaired_tot_bytes',
             'repaired_acc_bytes',
             'repaired_acc_packets',

             'fec_tot_packets',
             'fec_tot_bytes',
             'fec_acc_bytes',
             'fec_acc_packets',
            ];


In [321]:
from shutil import copyfile
import os.path
import pandas as pd
import datetime

#Make owd_statlogs
def make_merged_packetlogs(snd_packetlogs, rcv_packetlogs, merged_packetlogs_path):
    packetlog_columns     = get_packetlog_columns()
    snd_packets = pd.read_csv(snd_packetlogs, names=packetlog_columns)
    rcv_packets = pd.read_csv(rcv_packetlogs, names=packetlog_columns)
    
    result = pd.merge(snd_packets, rcv_packets, on='extended_seq', \
                           suffixes=['_snd','_rcv'])

    result.to_csv(merged_packetlogs_path, sep=',')
    
    #Make owd_statlogs
def make_path_statlogs(veth_logfile, 
#                        targets_logfile,
                       snd_statlogfile, 
                       rcv_statlogfile,
                       delta_packetlogfile, 
                       result_filepath):
    
    statlog_columns     = get_statslog_columns()
    
    delta_columns = get_delta_packetlog_columns()
   
    #targets_stats = pd.read_csv(targets_logfile,     names=["target"])
    veth_stats    = pd.read_csv(veth_logfile,        names=["tcps","bw"])
    snd_stats     = pd.read_csv(snd_statlogfile,     names=statlog_columns)
    rcv_stats     = pd.read_csv(rcv_statlogfile,     names=statlog_columns)
    packet_dstats = pd.read_csv(delta_packetlogfile, names=delta_columns)
    
    sending_rates = snd_stats.apply(lambda record: record['rcved_acc_packets'] * 28 + 
                                    record['rcved_acc_bytes'], axis=1)
    # sending_rates = snd_stats.apply(lambda record: record['rcved_acc_bytes'], axis=1)
    
    fec_rates     = snd_stats.apply(lambda record: record['fec_acc_packets'] * 28 + 
                                    record['fec_acc_bytes'], axis=1)
    
    ffre          = rcv_stats.apply(lambda record:  
                                    record['repaired_acc_packets'] / (record['repaired_acc_packets'] + record['lost_acc_packets']) if 0 < record['repaired_acc_packets'] else 0, axis=1)
    
    #print(rcv_stats['lost_acc_packets'])
    #print(rcv_stats['repaired_acc_packets'])
    #print(packet_dstats);
    result = pd.concat([veth_stats["bw"], 
                        sending_rates,
                        fec_rates,
                        packet_dstats['owd'],
                        packet_dstats['BiF'],
                        ffre,
#                         targets_stats["target"],
                        packet_dstats['playout_delay'],
                        rcv_stats["lost_acc_packets"],
                        rcv_stats["repaired_acc_packets"],
                       ], 
                       axis=1)
#    print(result)
    result.to_csv(result_filepath, sep=',')



In [338]:
from shutil import copyfile
import os.path
import pandas as pd
import datetime

def evaluate_statslog(statslog_file, 
                      packetlogs_file, 
                      veth_logfile, 
                      dpck_logfile,
                      sampling_num):
    packetlog_columns = get_packetlog_columns()
    statlog_table     = pd.read_csv(statslog_file, names=get_statslog_columns()).head(sampling_num + 1).tail(sampling_num)
    veth_table        = pd.read_csv(veth_logfile,  names=["tcps","bw"]).head(sampling_num + 1).tail(sampling_num)
    dpcklog_table     = pd.read_csv(dpck_logfile,  names=get_delta_packetlog_columns()).head(sampling_num + 1).tail(sampling_num)
    goodput_helper    = pd.Series(((statlog_table["rcved_acc_bytes"] - statlog_table["discarded_acc_bytes"]) + 
                                 (statlog_table["rcved_acc_packets"] - statlog_table["discarded_acc_packets"]) * 28 ) / 125)
    
    owdmean_helper    = pd.Series(dpcklog_table["owd"] / 1000)

    rcved_packets     = statlog_table.iloc[599]["rcved_tot_packets"]
    packetlogs_table  = pd.read_csv(packetlogs_file, names=packetlog_columns).head(rcved_packets + 1).tail(rcved_packets)
    
    repaired_packets  = statlog_table.iloc[599]["repaired_tot_packets"]
    rcved_packets     = statlog_table.iloc[599]["rcved_tot_packets"]
    rcved_bytes       = statlog_table.iloc[599]["rcved_tot_bytes"]
    discarded_packets = statlog_table.iloc[599]["discarded_tot_packets"]
    discarded_bytes   = statlog_table.iloc[599]["discarded_tot_bytes"]
    lost_packets      = statlog_table.iloc[599]["lost_tot_packets"]
    received_packets  = statlog_table.iloc[599]["rcved_tot_packets"]
    lost_rate         = float(lost_packets) / float(received_packets + lost_packets)

    frame_is_lost   = False
    lost_frames     = 0
    received_frames = 0    
    last_timestamp  = 0;
    
    for index, row in packetlogs_table.iterrows():
        if(row["timestamp"] != 0):
            last_timestamp = row["timestamp"]
            break

    for index, row in packetlogs_table.iterrows():
        if(row["timestamp"] == 0 or row["state"] != 1):
            frame_is_lost = True;
            continue;
        if(row["timestamp"] != last_timestamp):
            last_timestamp = row["timestamp"]
            if(frame_is_lost):
                lost_frames += 1
            else: 
                received_frames += 1
            frame_is_lost = False;
    
    return{
        'repaired_packets_num':    repaired_packets,
        'repaired_rate':           float(repaired_packets) / float(repaired_packets + lost_packets),
        'lost_packet_rate':        lost_rate, 
        'goodput_mean':            goodput_helper.mean(),
        'lost_packets_num':        lost_packets,
        'rcved_packets_num':       rcved_packets,
        'lost_frames_num':         lost_frames,
        'lost_frame_rate':         float(lost_frames) / float(lost_frames + received_frames),
        'received_frames':         received_frames,
        'owd mean':                owdmean_helper.mean(),
        'owd std':                 owdmean_helper.std(),
    };

def evaluate_videofiles(original, processed, width, height, framesnum, targetname):
    psnr_file   = targetname + "_psnr.csv"
    ssim_file   = targetname + "_ssim.csv"
    msssim_file = targetname + "_msssim.csv"
    vifp_file   = targetname + "_vifp.csv"
    columns     = ["frame","value"]
    
    print("Evaluating Video")
    print("Source: " + original +" Processed: " + processed + " W: " + str(width) + " H: " + str(height) + " psnr: " + psnr_file)
    print("Framesnum: " + str(framesnum))
    
    !vqmt $original $processed $height $width $framesnum 1 $targetname PSNR SSIM MSSSIM VIFP
    
    psnr    = pd.Series(pd.read_csv(psnr_file,    names=columns).head(-1).tail(-1).astype('float')["value"])
    ssim    = pd.Series(pd.read_csv(ssim_file,    names=columns).head(-1).tail(-1).astype('float')["value"])
    msssim  = pd.Series(pd.read_csv(msssim_file,  names=columns).head(-1).tail(-1).astype('float')["value"])
    vifp    = pd.Series(pd.read_csv(vifp_file,    names=columns).head(-1).tail(-1).astype('float')["value"])
    
    return {
        "psnr_mean": psnr.mean(),
        "psnr_std":  psnr.std(),
        "ssim_mean": ssim.mean(),
        "ssim_std":  ssim.std(),
        "msssim_mean": msssim.mean(),
        "msssim_std":  msssim.std(),
        "vifp_mean": vifp.mean(),
        "vifp_std":  vifp.std(),
        };
    
    

In [339]:
#RMCAT 1 analyzer
import csv

#!cd ../../temp; ls #The directory where the logfiles are
base_path       = "../.."
meas_path       = base_path + "/temp"
plot_path       = "../gnuplots"
path_delay_str  = "100000" #in us

snd_statlogs    = meas_path + "/snd_statlogs.csv"
rcv_statlogs    = meas_path + "/rcv_statlogs.csv"
snd_packetlogs  = meas_path + "/snd_packetlogs.csv"
rcv_packetlogs  = meas_path + "/rcv_packetlogs.csv"
bandwidth       = meas_path + "/bandwidth.csv"
veth0_stats     = meas_path + "/veth0_stats.csv"
merged_statlogs = meas_path + "/merged_statlogs.csv"
delta_statlogs  = meas_path + "/delta_statlogs.csv"
rmcat0_statlogs = meas_path + "/rmcat0_statlogs.csv"
rmcat0_tmplogs  = meas_path + "/rmcat0_tmplogs.csv"
outplot         = meas_path + "/statlogs.pdf"
targetsfile     = meas_path + "/targets.csv"
evalfile        = meas_path + "/evaluation.csv"
plotfile        = plot_path + "/rmcat1.plot"




#Made OWD statlogs
make_merged_packetlogs(snd_packetlogs, rcv_packetlogs, merged_statlogs)
!./../../make_delta_statlogs $merged_statlogs $delta_statlogs

make_path_statlogs(veth0_stats, 
#                    targetsfile, 
                   snd_statlogs, 
                   rcv_statlogs, 
                   delta_statlogs, 
                   rmcat0_statlogs)

!sed '1d' $rmcat0_statlogs > $rmcat0_tmplogs; mv $rmcat0_tmplogs $rmcat0_statlogs

!gnuplot -e "statlogs='$rmcat0_statlogs'" \
         -e "output_file='$outplot'" \
         -e "path_delay=50000" \
         $plotfile

eval_result = evaluate_statslog(rcv_statlogs, rcv_packetlogs, veth0_stats, delta_statlogs, 600)
print(eval_result)

w = csv.writer(open(evalfile, "w"))
for key, val in eval_result.items():
    w.writerow([key, val])
    



Make Delta Packet Statlogs.
Source file:             ../../temp/merged_statlogs.csv
Destination file:        ../../temp/delta_statlogs.csv
Accumulation length:     1000ms
Sampling time:           100ms
Unit of delay:           us
Unit of bytes in flight: byte
Delta PacketStatlog is made
{'received_frames': 1410, 'goodput_mean': 820.05355999999995, 'rcved_packets_num': 5008, 'lost_frames_num': 8, 'repaired_rate': 0.0, 'owd mean': 115.25631833333333, 'owd std': 46.82324440493921, 'repaired_packets_num': 0, 'lost_frame_rate': 0.005641748942172073, 'lost_packets_num': 137, 'lost_packet_rate': 0.02662779397473275}


In [336]:

originalvideo   = base_path + "/produced.yuv"
processedvideo  = base_path + "/consumed.yuv"

videoeval_result = evaluate_videofiles(originalvideo, processedvideo, 352, 288, 
                    eval_result["lost_frames_num"] + eval_result["received_frames"], meas_path + "vqmt")

print(videoeval_result)
w = csv.writer(open(evalfile, "a"))
for key, val in videoeval_result.items():
    w.writerow([key, val])
    

save_destination = base_path + "/target"
if(os.path.exists(save_destination) == False): 
    !mkdir $save_destination
!cp $meas_path/* $save_destination

Evaluating Video
Source: ../../produced.yuv Processed: ../../consumed.yuv W: 352 H: 288 psnr: ../../tempvqmt_psnr.csv
Framesnum: 1418
Time: 117.307s
{'msssim_std': 0.32795451075585247, 'vifp_mean': 0.13468774400564176, 'vifp_std': 0.20689037416352749, 'psnr_std': 8.2379051378584442, 'psnr_mean': 18.675446601551478, 'msssim_mean': 0.35613744146685472, 'ssim_std': 0.21055099417997239, 'ssim_mean': 0.48980704866008462}


In [337]:
read = {}
for key, val in csv.reader(open(evalfile)):
    read[key] = val
print("Checking weather " + videoevalfile + " contains the evaluation or not");
print(read)

save_destination = base_path + "/target"
if(os.path.exists(save_destination) == False): 
    !mkdir $save_destination
!cp $meas_path/* $save_destination


Checking weather ../../temp/videoeval.csv contains the evaluation or not
{'rcved_packets_num': '5008', 'lost_frames_num': '8', 'msssim_std': '0.32795451075585247', 'repaired_rate': '0.0', 'owd mean': '115256.31833333333', 'ssim_std': '0.21055099417997239', 'msssim_mean': '0.35613744146685472', 'vifp_mean': '0.13468774400564176', 'owd std': '46823.244404939214', 'ssim_mean': '0.48980704866008462', 'repaired_packets_num': '0', 'received_frames': '1410', 'goodput_mean': '820.05355999999995', 'psnr_mean': '18.675446601551478', 'lost_frame_rate': '0.005641748942172073', 'vifp_std': '0.20689037416352749', 'lost_packets_num': '137', 'lost_packet_rate': '0.02662779397473275', 'psnr_std': '8.2379051378584442'}
